In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import spotipy
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:
r = requests.get('https://www.billboard.com/charts/hot-100/').content

soup = BeautifulSoup(r, 'html.parser')

top1_artist = soup.find_all('span', attrs={'class': 'c-label a-no-trucate a-font-primary-s \
lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 \
lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet'})

top1_song = soup.find_all('h3', attrs={"class": "c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 u-font-size-23@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-245 u-max-width-230@tablet-only u-letter-spacing-0028@tablet"})

songs = soup.find_all("h3", attrs={"class": "c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only"})

song_full = top1_song + songs

artists = soup.find_all("span", attrs={"class": "c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only"})

artists_full = top1_artist + artists




In [3]:
artist_names_list = []
for artist in artists_full:
    artist_names_list.append(artist.get_text(strip=True))

In [4]:
song_names_list = []
for song in songs:
    song_names_list.append(song.get_text(strip=True))


In [5]:
billboard100 = zip(artist_names_list, song_names_list)
billboard100_df = pd.DataFrame(billboard100, columns=["artist name", "song name"])

In [14]:
expanded_df = pd.DataFrame(columns=billboard100_df.columns)

def split_artist_name(name, delimiters):
    if not delimiters:
        return [name]
    delimiter = delimiters[0]
    split_names = name.split(delimiter)
    results = []
    for split_name in split_names:
        results.extend(split_artist_name(split_name.strip(), delimiters[1:]))
    return results

for _, row in billboard100_df.iterrows():
    # Use recursive function to split by "Featuring", "&", "With", ",", and " x "
    delimiters = [' Featuring ', ' & ', ' With ', ', ', ' x ']
    artists = split_artist_name(row['artist name'], delimiters)
    
    for artist in artists:
        expanded_df = expanded_df.append({
            'artist name': artist.strip(),
            'song name': row['song name']
        }, ignore_index=True)

In [15]:
expanded_df.head(50)

,artist name,song name
0,Olivia Rodrigo,Paint The Town Red
1,Doja Cat,I Remember Everything
2,Zach Bryan,Fast Car
3,Kacey Musgraves,Fast Car
4,Luke Combs,Cruel Summer
5,Taylor Swift,Last Night
6,Morgan Wallen,Bad Idea Right?
7,Olivia Rodrigo,Snooze
8,SZA,Fukumean
9,Gunna,Dance The Night


In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="57fb64fab0af4711bef89b43ecac3361",
                                                           client_secret="8bb73ad9b5f94d05a85dbcd27039103c"))

In [16]:
def get_track_id(artist_name, track_name):
    query = f'artist:{artist_name} track:{track_name}'
    result = sp.search(q=query, type='track', limit=1)
    
    if result['tracks']['items']:
        return result['tracks']['items'][0]['id']
    else:
        return None


In [17]:
# Assuming you already have 'expanded_df'
expanded_df['Spotify_ID'] = expanded_df.apply(lambda row: get_track_id(row['artist name'], row['song name']), axis=1)


In [18]:
def get_audio_features(track_id):
    if track_id:
        return sp.audio_features([track_id])[0]
    return None

expanded_df['Audio_Features'] = expanded_df['Spotify_ID'].apply(get_audio_features)


In [19]:
expanded_df['Exists_on_Spotify'] = ~expanded_df['Audio_Features'].isnull()


In [24]:
expanded_df['Exists_on_Spotify'].sum()

12

In [ ]:
def get_audio_features(artist):
    # get tracks from artist
    results = sp.search(q=f'artist:{artist}', limit=50)
    
    # extract the track ids
    track_ids = [track['id'] for track in results['tracks']['items']]
    song_names = [track['name'] for track in results['tracks']['items']]
    
    # extract the audio features
    audio_features = sp.audio_features(track_ids)
    
    # filter out None values from audio_features
    audio_features = [af for af in audio_features if af]
    if not audio_features:
        return None  # if all audio features are None, return None

    # store audio features in a dataframe
    df = pd.DataFrame(audio_features)
    df['artist'] = artist
    df['song name'] = song_names[:len(audio_features)]  # take only song names corresponding to valid audio features
    return df

In [ ]:
artists = list(expanded_df['artist name'].unique())
songs = list(expanded_df['song name'].unique())

In [ ]:
df = pd.DataFrame()

for artist in artists:
    df_artist = get_audio_features(artist)
    if df_artist is not None:  # only append if there are valid audio features
        df = pd.concat([df, df_artist])

df = df.reset_index(drop=True)
df